In [1]:
import os
#MODEL = "mixtral:8x7b"
MODEL = "llama2"

In [2]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("mixtral:8x7b"):
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings()
else:
     model = Ollama(model=MODEL)
     embeddings = OllamaEmbeddings()
model.invoke("tell me a joke")

"\nSure! Here's a classic one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that made you smile!"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Climate Change.pdf")
pages = loader.load_and_split()
pages

[Document(page_content="Climate Change  \nClimate change is one of the most significant and pressing issues of our time, \naffecting every aspect of our lives, from the environment to the economy, \nhealth, and security. It refers to the long -term alteration of temperature and \ntypical weather patterns  in a place. While climate change is a natural \nphenomenon, human activities, particularly since the Industrial Revolution, \nhave significantly accelerated its pace, leading to unprecedented and \npotentially catastrophic changes in the Earth's climate system.  \n \nThe scientific consensus on climate change is clear: the Earth's climate is \nwarming, and human activities are the primary driver. The Intergovernmental \nPanel on Climate Change (IPCC), a leading authority on climate science, has \nreported that global tempera tures have risen by approximately 1.2 degrees \nCelsius (2.2 degrees Fahrenheit) since the late 19th century. This warming \ntrend is largely due to the increased

In [6]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="1.2 degrees Celsius.", question="How much have global temperatures risen since the late 19th century?"))

Human: 
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context:  1.2 degrees Celsius.

Question: How much have global temperatures risen since the late 19th century?



In [7]:
chain = prompt | model | parser

In [8]:
chain.invoke({
    "context": "1.2 degrees Celsius",
    "question": "How much have global temperatures risen since the late 19th century?"
})

'Sure, I\'d be happy to help! Based on the context you provided, the answer to the question "How much have global temperatures risen since the late 19th century?" is:\n\nAccording to NASA, the average global temperature has risen by about 1.2 degrees Celsius (2.2 degrees Fahrenheit) since the late 19th century. This is based on data from weather stations, satellite measurements, and other sources.'

In [9]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [10]:
from langchain_community.vectorstores import DocArrayInMemorySearch
import docarray
# Assuming pages and embeddings are already defined
vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)


In [11]:
retriever = vectorstore.as_retriever()

vectorstore.as_retriever().invoke("climate")

[Document(page_content='organizations can play a vital role in raising awareness about the causes and \nimpacts of climate change and promoting actions that individuals can take to \nreduce their carbon footprint. This includes simple measures such as reducing \nenergy consumption, usi ng public transportation, recycling, and supporting \nsustainable food practices.  \n \nBehavioral and cultural shifts are essential for achieving long -term climate \ngoals. Individuals can make a significant impact by adopting sustainable \nlifestyle choices, such as reducing meat consumption, minimizing waste, and \nsupporting renewable energy. B usinesses also have a critical role to play in \naddressing climate change. Corporate sustainability initiatives, such as reducing \ngreenhouse gas emissions, sourcing renewable energy, and implementing \ncircular economy practices, can significantly contribute to global climate \nefforts.  \n \nInnovation and technology are at the heart of addressing climate

In [12]:
from operator import itemgetter
from langchain.schema.runnable import RunnableMap, RunnablePassthrough, RunnableLambda
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [13]:
# Assuming the chain function and other necessary imports are defined as previously discussed

# Example questions
questions = [
    "What is the greenhouse effect?",
    "What is the primary driver of recent climate change?"
]

# Iterate through the questions and process each one with the chain function
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the greenhouse effect?
Answer: The greenhouse effect is a natural phenomenon where certain gases in the Earth's atmosphere, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), trap heat from the sun and prevent it from escaping back into space. This trapped heat warms the Earth's surface and atmosphere, contributing to the planet's overall temperature.

The greenhouse effect is important for maintaining a habitable climate on Earth. Without it, the planet would be much colder and less hospitable to life. However, the increased concentration of greenhouse gases in the atmosphere due to human activities such as burning fossil fuels, deforestation, and agriculture has accelerated the greenhouse effect, leading to unprecedented and potentially catastrophic changes in the Earth's climate system.

The main greenhouse gases are:

1. Carbon dioxide (CO2): The most abundant greenhouse gas, released through human activities such as burning fossil fuels, defore

In [14]:
chain.batch([{"question": q} for q in questions])

["The greenhouse effect is a natural phenomenon where greenhouse gases in the Earth's atmosphere trap heat from the sun, warming the planet. The scientific consensus is clear: the Earth's climate is warming, and human activities are the primary driver of this warming trend. The Intergovernmental Panel on Climate Change (IPCC) reports that global temperatures have risen by approximately 1.2 degrees Celsius (2.2 degrees Fahrenheit) since the late 19th century, primarily due to the increased concentration of greenhouse gases in the atmosphere, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O). These gases trap heat from the sun and prevent it from escaping back into space.\n\nThe burning of fossil fuels for energy and transportation is the largest source of greenhouse gas emissions, followed by deforestation, agriculture, and industrial processes. The increase in greenhouse gases enhances the natural greenhouse effect, leading to a warming of the Earth's surface and atm